In [1]:
import pandas as pd
import kagglehub
import os


In [2]:
companies = pd.read_csv('raw_linkedin_data/companies/companies.csv')
employee_counts = pd.read_csv('raw_linkedin_data/companies/employee_counts.csv')
industries = pd.read_csv('raw_linkedin_data/companies/company_industries.csv')
specialities = pd.read_csv('raw_linkedin_data/companies/company_specialities.csv')

industries_ids = pd.read_csv('raw_linkedin_data/mappings/industries.csv')

In [3]:
#did a simple check to see if there are duplicates in the data regarding industries
#this is important because if there are duplicates, we need to process them
#in order to avoid any errors in the data

companies_has_duplicates = companies['company_id'].duplicated().any()
print("Duplicates present in companies.csv:", companies_has_duplicates)

employee_counts_has_duplicates = employee_counts['company_id'].duplicated().any()
print("Duplicates present in employee_counts.csv:", employee_counts_has_duplicates)

industry_has_duplicates = industries['company_id'].duplicated().any()
print("Duplicates present in company_industries.csv:", industry_has_duplicates)

specialities_has_duplicates = specialities['company_id'].duplicated().any()
print("Duplicates present in company_specialities.csv:", specialities_has_duplicates)


Duplicates present in companies.csv: False
Duplicates present in employee_counts.csv: True
Duplicates present in company_industries.csv: True
Duplicates present in company_specialities.csv: True


In [4]:
employee_counts['time_recorded'] = pd.to_datetime(employee_counts['time_recorded'])
employee_counts_sorted = employee_counts.sort_values(['company_id', 'time_recorded'], ascending=[True, False])
employee_counts_unique = employee_counts_sorted.drop_duplicates(subset='company_id', keep='first')
# Convert 'time_recorded' to datetime format
# Sort the employee_counts dataframe by 'company_id' and 'record_date' in descending order
# Drop duplicates, keeping the first occurrence (most recent record) for each 'company_id'


In [5]:
#check to the earlier step to see if we are getting the most recent tie
max_dates = employee_counts.groupby('company_id')['time_recorded'].max().reset_index().rename(columns={'time_recorded': 'max_date'})
merged = employee_counts_unique.merge(max_dates, on='company_id')
merged['is_most_recent'] = merged['time_recorded'] == merged['max_date']
all_recent = merged['is_most_recent'].all()
print("All records are the most recent:", all_recent)

All records are the most recent: True


In [6]:
industries_with_ids = industries.merge(
    industries_ids,
    left_on='industry',
    right_on='industry_name',
    how='left'
)

# Drop the original industry name columns (both from industries and the mapping)
industries_with_ids = industries_with_ids.drop(columns=['industry', 'industry_name'])

# Convert non-NaN values in 'industry_id' to int while leaving NaN values unchanged
industries_with_ids['industry_id'] = industries_with_ids['industry_id'].apply(
    lambda x: int(x) if pd.notnull(x) else x
)

print(industries_with_ids)


       company_id  industry_id
0          391906         82.0
1        22292832         48.0
2           20300         41.0
3         3570660         82.0
4          878353        104.0
...           ...          ...
24370       32313        143.0
24371    15225088         96.0
24372     2852377         31.0
24373    19114724         48.0
24374     8060959        116.0

[24375 rows x 2 columns]


In [7]:
specialities['speciality'] = specialities['speciality'].astype(str).str.strip()

aggregated_industries_list = industries_with_ids.groupby('company_id')['industry_id'].apply(
    lambda x: sorted([i for i in list(x.unique()) if pd.notnull(i)])  # Filter out NaN values
).reset_index()

aggregated_specialities_list = specialities.groupby('company_id')['speciality'].apply(
    lambda x: sorted(list(x.unique()))
).reset_index()

In [8]:
def check_list_aggregation(df_original, df_aggregated, key_col, value_col, list_col):
    errors = []
    for idx, row in df_aggregated.iterrows():
        company_id = row[key_col]
        # Get unique values from the original dataframe for this company
        original_values_with_nan = list(df_original[df_original[key_col] == company_id][value_col].unique())
        # Filter out NaN values to match the aggregation logic
        original_values = sorted([val for val in original_values_with_nan if pd.notnull(val)])
        
        # Get the aggregated list
        aggregated_values = sorted(row[list_col])
        
        if aggregated_values != original_values:
            errors.append((company_id, original_values, aggregated_values))
    
    if errors:
        print(f"Errors found in aggregation ({len(errors)} companies):")
        for comp_id, orig, agg in errors[:5]:  # Print first 5 errors to avoid excessive output
            print(f"Company ID {comp_id}: Expected {orig}, Aggregated {agg}")
        if len(errors) > 5:
            print(f"...and {len(errors) - 5} more errors.")
    else:
        print(f"Aggregation is correct for all companies in column '{list_col}'.")

# Check aggregation for industries
check_list_aggregation(
    industries_with_ids,            # Original dataframe
    aggregated_industries_list,    # Aggregated dataframe
    'company_id',                  # Key column
    'industry_id',                    # Value column in original data
    'industry_id'                     # List column in aggregated data (before renaming)
)

# Check aggregation for specialities
check_list_aggregation(
    specialities,          # Original dataframe
    aggregated_specialities_list,  # Aggregated dataframe
    'company_id',                  # Key column
    'speciality',                  # Value column in original data
    'speciality'                   # List column in aggregated data (before renaming)
)


Aggregation is correct for all companies in column 'industry_id'.
Aggregation is correct for all companies in column 'speciality'.


In [9]:
final_company_data = companies.merge(employee_counts_unique, on='company_id', how='left') \
                        .merge(aggregated_industries_list, on='company_id', how='left') \
                        .merge(aggregated_specialities_list, on='company_id', how='left')

In [10]:
print(final_company_data.isnull().sum())


company_id           0
name                 1
description        297
company_size      2774
state               22
country              0
city                 1
zip_code            28
address             22
url                  0
employee_count       0
follower_count       0
time_recorded        0
industry_id        108
speciality        6693
dtype: int64


In [11]:
def check_missing_consistency(company_data, original_df, column_name):
    """
    Returns True if all companies with NaN in company_data for the given column
    are absent in the original_df (i.e., they have no record in the original source).
    
    Parameters:
    - company_data: DataFrame that contains the aggregated column.
    - original_df: DataFrame of the original data (industries or specialties).
    - column_name: Name of the aggregated column in company_data ('industry' or 'speciality').
    
    Returns:
    - Boolean: True if consistency holds, False otherwise.
    """
    # Get the set of company_ids with missing aggregated values
    missing_ids = set(company_data[company_data[column_name].isna()]['company_id'])
    # Get the set of company_ids that appear in the original data
    original_ids = set(original_df['company_id'].unique())
    
    # If a company has a missing value in the aggregated column but appears in the original data,
    # that's a problem.
    return len(missing_ids.intersection(original_ids)) == 0

def check_missing_industries_and_specialties(company_data, industries_df, specialities_df):
    """
    Checks missing data consistency for both industries and specialities columns.
    
    Returns True if both checks are True, otherwise False.
    """
    industry_check = check_missing_consistency(company_data, industries_df, 'industry_id')
    speciality_check = check_missing_consistency(company_data, specialities_df, 'speciality')
    return industry_check and speciality_check

# Example usage:
result = check_missing_industries_and_specialties(final_company_data, industries_with_ids, specialities)
print("Missing values are consistent with the original data:", result)


Missing values are consistent with the original data: True


In [12]:
#clean the postings dataframe

postings = pd.read_csv('raw_linkedin_data/postings.csv')
jobs_industries = pd.read_csv('raw_linkedin_data/jobs/job_industries.csv')
job_salaries = pd.read_csv('raw_linkedin_data/jobs/salaries.csv')
job_skills = pd.read_csv('raw_linkedin_data/jobs/job_skills.csv')

benefits = pd.read_csv('raw_linkedin_data/jobs/benefits.csv')

In [13]:
#has a salaries.csv file that contains the salary information for each job posting and postings.csv file that contains the job postings with salary information
#want to make sure that the salary is 

def are_salaries_consistent(postings, salaries):
    postings['job_id'] = postings['job_id'].astype(str)
    salaries['job_id'] = salaries['job_id'].astype(str)
    
    postings_with_salary = postings[postings['max_salary'].notna() | postings['min_salary'].notna() | postings['med_salary'].notna()]
    
    posting_job_ids = set(postings_with_salary['job_id'])    
    salary_job_ids = set(salaries['job_id'])
    
    if not posting_job_ids.issubset(salary_job_ids):
        return False
    
    fields = ['max_salary', 'med_salary', 'min_salary', 'pay_period', 'currency', 'compensation_type']
    
    relevant_salaries = salaries[salaries['job_id'].isin(posting_job_ids)]
    relevant_salaries = relevant_salaries.set_index('job_id')
    
    for _, post in postings_with_salary.iterrows():
        job_id = post['job_id']
        salary = relevant_salaries.loc[job_id]
        
        for field in fields:
            if pd.isna(post[field]) and pd.isna(salary[field]):
                continue
                
            if field in ['max_salary', 'med_salary', 'min_salary']:
                if not pd.isna(post[field]) and not pd.isna(salary[field]):
                    if abs(float(post[field]) - float(salary[field])) > 0.01:
                        return False
            elif str(post[field]).strip() != str(salary[field]).strip():
                return False
    
    return True

print("salary information in salaries.csv and posting.csv is the same:", are_salaries_consistent(postings, job_salaries))

salary information in salaries.csv and posting.csv is the same: True


In [14]:
def create_salary_dataset(postings):
    salary_df = postings.copy()
    
    has_salary = (
        salary_df['max_salary'].notna() | 
        salary_df['med_salary'].notna() | 
        salary_df['min_salary'].notna() |
        salary_df['normalized_salary'].notna()
    )
    
    has_period = salary_df['pay_period'].notna()
    
    salary_df = salary_df[has_salary & has_period]
    
    return salary_df

postings_with_salary = create_salary_dataset(postings)
print(f"Original postings: {len(postings)}")
print(f"Postings with salary information: {len(postings_with_salary)}")
print(f"Percentage with salary info: {len(postings_with_salary)/len(postings)*100:.2f}%")

Original postings: 123849
Postings with salary information: 36073
Percentage with salary info: 29.13%


In [15]:
all_usd = (postings_with_salary['currency'] == 'USD').all()
print(f"All salaries are in USD: {all_usd}")

# If you want to see the count of different currencies:
currency_counts = postings_with_salary['currency'].value_counts()
print(currency_counts)

All salaries are in USD: False
currency
USD    36058
EUR        6
CAD        3
BBD        2
AUD        2
GBP        2
Name: count, dtype: int64


In [16]:
non_usd_jobs = postings_with_salary[postings_with_salary['currency'] != 'USD']
print("\nID\tLocation\tCurrency")
print("_"*50)
for _, job in non_usd_jobs.iterrows():
    print(f"{job['job_id']}\t{job['location']}\t{job['currency']}")



ID	Location	Currency
__________________________________________________
3885870216	United States	CAD
3888974657	Reno, NV	CAD
3888974658	Kensett, AR	BBD
3888976416	Huntsville, AL	BBD
3889710534	Germany, PA	EUR
3889711148	Germany, PA	EUR
3891804969	New York, United States	AUD
3894295381	Whippany, NJ	CAD
3901991613	New York, United States	AUD
3902347641	Ennis, TX	EUR
3903439951	Center City, MN	EUR
3903446331	England, AR	GBP
3903449123	England, AR	GBP
3903818935	Center City, MN	EUR
3903824359	Center City, MN	EUR


In [17]:
#Since all the jobs that are not in USD are located in United States, we should convert the salaries to USD using these conversion rates.
#1.00 US Dollar = 0.92367131 EUR
#1.00 US Dollar = 1.4352374 CAD
#1.00 US Dollar = 2.00 BBD
#1.00 US Dollar = 1.594355 AUD
#1.00 US Dollar = 0.77381294 GBP


# Define conversion rates to USD (multiply by these to get USD)
currency_to_usd = {
    'EUR': 1 / 0.92367131,
    'CAD': 1 / 1.4352374,
    'BBD': 1 / 2.00,
    'AUD': 1 / 1.594355,
    'GBP': 1 / 0.77381294,
    'USD': 1.0  # No conversion needed
}

# Function to convert salary to USD
def convert_to_usd(row):
    if row['currency'] != 'USD':
        conversion_rate = currency_to_usd.get(row['currency'], 1.0)
        
        # Convert all salary fields including normalized_salary
        for field in ['max_salary', 'med_salary', 'min_salary', 'normalized_salary']:
            if field in row and pd.notna(row[field]):
                row[field] = row[field] * conversion_rate
        
        # Update currency to USD
        row['currency'] = 'USD'
    
    return row

# Apply conversion to all rows
postings_with_salary = postings_with_salary.apply(convert_to_usd, axis=1)

# Verify all currencies are now USD
all_usd = (postings_with_salary['currency'] == 'USD').all()
print(f"All salaries converted to USD: {all_usd}")

All salaries converted to USD: True


In [18]:
unique_work_types = postings_with_salary['formatted_work_type'].unique()
print(unique_work_types)

unique_pay_periods = postings_with_salary['pay_period'].unique()
print(unique_pay_periods)

hourly_by_work_type = postings_with_salary[postings_with_salary['pay_period'] == 'HOURLY']['formatted_work_type'].value_counts()

print("\n\nCount of hourly jobs by work type:")
print(hourly_by_work_type)

['Full-time' 'Internship' 'Contract' 'Part-time' 'Temporary' 'Other'
 'Volunteer']
['HOURLY' 'YEARLY' 'MONTHLY' 'WEEKLY' 'BIWEEKLY']


Count of hourly jobs by work type:
formatted_work_type
Full-time     8819
Contract      3343
Part-time     1955
Temporary      332
Internship     202
Other           90
Name: count, dtype: int64


In [19]:


#after carefully examining the data, we can see that the normalized salary is sometimes taking the med_salary when available, sometimes average of max and min, and sometimes the max_salary when the med_salary is not available. 
#this column might not be of use to use since it is not consistent. 

# Drop the 'normalized_salary' column
postings_with_salary = postings_with_salary.drop(columns=['normalized_salary'])


In [20]:
def create_normalized_annual_salaries(df):
    # Create new columns
    df['norm_min_annual'] = df['min_salary'].copy()
    df['norm_med_annual'] = df['med_salary'].copy()
    df['norm_max_annual'] = df['max_salary'].copy()
    
    # Define multipliers for each pay period
    pay_period_multipliers = {
        'HOURLY': lambda row: 20 * 52 if row['work_type'] == 'PART_TIME' else 40 * 52,
        'WEEKLY': 52,
        'BIWEEKLY': 26,
        'MONTHLY': 12,
        'YEARLY': 1,
        'ANNUAL': 1
    }
    
    # Apply conversions
    for idx, row in df.iterrows():
        if row['pay_period'] in pay_period_multipliers:
            multiplier = pay_period_multipliers[row['pay_period']]
            if callable(multiplier):
                multiplier = multiplier(row)
                
            for col in ['norm_min_annual', 'norm_med_annual', 'norm_max_annual']:
                if pd.notna(row[col]):
                    df.at[idx, col] = row[col] * multiplier
    
    # Fill missing med_salary with average of min and max
    mask = pd.isna(df['norm_med_annual']) & pd.notna(df['norm_min_annual']) & pd.notna(df['norm_max_annual'])
    df.loc[mask, 'norm_med_annual'] = (df.loc[mask, 'norm_min_annual'] + df.loc[mask, 'norm_max_annual']) / 2
    
    return df

postings_with_salary = create_normalized_annual_salaries(postings_with_salary)

In [21]:
import re
non_standard_locations = []
for loc in postings_with_salary['location'].dropna().unique():
   if not re.search(r', [A-Z]{2}$', str(loc)):
       non_standard_locations.append(loc)

# Sort and print them
non_standard_locations.sort()
print(len(non_standard_locations))

print(f"Locations without standard City, State format ({len(non_standard_locations)} found):")
for loc in non_standard_locations:
   print(f"- {loc}")

240
Locations without standard City, State format (240 found):
- Alabama, United States
- Alaska, United States
- Albany, New York Metropolitan Area
- Albuquerque, New Mexico, United States
- Albuquerque-Santa Fe Metropolitan Area
- Anchorage, Alaska, United States
- Ann Arbor, Michigan, United States
- Appleton-Oshkosh-Neenah Area
- Arizona, United States
- Arkansas, United States
- Atlanta Metropolitan Area
- Aurora, Colorado, United States
- Austin, Texas Metropolitan Area
- Austin, Texas, United States
- Baton Rouge Metropolitan Area
- Beaumont-Port Arthur Area
- Bellingham Metropolitan Area
- Bellingham, Washington, United States
- Bend, Oregon, United States
- Birkdale Village, North Carolina, United States
- Blacksburg, Virginia, United States
- Blacksburg-Christiansburg-Radford Area
- Boise Metropolitan Area
- Boston, Massachusetts, United States
- Brooklyn, New York, United States
- Broomfield, Colorado, United States
- Buffalo-Niagara Falls Area
- Calabasas, California, Unite

In [22]:
def standardize_locations(df):
    # Dictionary mapping state names to abbreviations 
    state_to_abbrev = {
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 
        'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
        'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 
        'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
        'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
        'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
        'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
        'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
        'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
        'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
        'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
        'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
        'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'
    }
    
    # Dictionary mapping metro areas to city, state format
    metro_to_city_state = {
    # A
    "Albany, New York Metropolitan Area": "Albany, NY",
    "Albuquerque-Santa Fe Metropolitan Area": "Albuquerque, NM",
    "Appleton-Oshkosh-Neenah Area": "Appleton, WI",
    "Atlanta Metropolitan Area": "Atlanta, GA",
    "Austin, Texas Metropolitan Area": "Austin, TX",
    
    # B
    "Baton Rouge Metropolitan Area": "Baton Rouge, LA",
    "Beaumont-Port Arthur Area": "Beaumont, TX",
    "Bellingham Metropolitan Area": "Bellingham, WA",
    "Blacksburg-Christiansburg-Radford Area": "Blacksburg, VA",
    "Boise Metropolitan Area": "Boise, ID",
    "Buffalo-Niagara Falls Area": "Buffalo, NY",
    
    # C
    "Cape Coral Metropolitan Area": "Cape Coral, FL",
    "Charleston, South Carolina Metropolitan Area": "Charleston, SC",
    "Charlotte Metro": "Charlotte, NC",
    "Cincinnati Metropolitan Area": "Cincinnati, OH",
    "College Station-Bryan Area": "College Station, TX",
    "Columbia, South Carolina Metropolitan Area": "Columbia, SC",
    "Columbus, Ohio Metropolitan Area": "Columbus, OH",
    "Crestview-Fort Walton Beach-Destin Area": "Fort Walton Beach, FL",
    
    # D
    "Dallas-Fort Worth Metroplex": "Dallas, TX",
    "Denver Metropolitan Area": "Denver, CO",
    "Des Moines Metropolitan Area": "Des Moines, IA",
    "Detroit Metropolitan Area": "Detroit, MI",
    
    # E
    "Eau Claire-Menomonie Area": "Eau Claire, WI",
    "Erie-Meadville Area": "Erie, PA",
    
    # F
    "Fayetteville, North Carolina Metropolitan Area": "Fayetteville, NC",
    
    # G
    "Grand Rapids Metropolitan Area": "Grand Rapids, MI",
    "Greater Albany, Georgia Area": "Albany, GA",
    "Greater Asheville": "Asheville, NC",
    "Greater Augusta Area": "Augusta, GA",
    "Greater Bend Area": "Bend, OR",
    "Greater Birmingham, Alabama Area": "Birmingham, AL",
    "Greater Bismarck Area": "Bismarck, ND",
    "Greater Bloomington Area": "Bloomington, IN",
    "Greater Boston": "Boston, MA",
    "Greater Burlington Area": "Burlington, VT",
    "Greater Chattanooga": "Chattanooga, TN",
    "Greater Chicago Area": "Chicago, IL",
    "Greater Chico Area": "Chico, CA",
    "Greater Cleveland": "Cleveland, OH",
    "Greater Colorado Springs Area": "Colorado Springs, CO",
    "Greater Corpus Christi Area": "Corpus Christi, TX",
    "Greater Dothan": "Dothan, AL",
    "Greater Enid Area": "Enid, OK",
    "Greater Eugene-Springfield Area": "Eugene, OR",
    "Greater Fayetteville, AR Area": "Fayetteville, AR",
    "Greater Flagstaff Area": "Flagstaff, AZ",
    "Greater Fort Collins Area": "Fort Collins, CO",
    "Greater Fort Wayne": "Fort Wayne, IN",
    "Greater Goldsboro Area": "Goldsboro, NC",
    "Greater Grand Junction Area": "Grand Junction, CO",
    "Greater Hartford": "Hartford, CT",
    "Greater Houston": "Houston, TX",
    "Greater Indianapolis": "Indianapolis, IN",
    "Greater Jackson, MI Area": "Jackson, MI",
    "Greater Lansing": "Lansing, MI",
    "Greater Lexington Area": "Lexington, KY",
    "Greater Macon": "Macon, GA",
    "Greater Madison Area": "Madison, WI",
    "Greater McAllen Area": "McAllen, TX",
    "Greater Milwaukee": "Milwaukee, WI",
    "Greater Minneapolis-St. Paul Area": "Minneapolis, MN",
    "Greater Morgantown Area": "Morgantown, WV",
    "Greater New Orleans Region": "New Orleans, LA",
    "Greater Orlando": "Orlando, FL",
    "Greater Philadelphia": "Philadelphia, PA",
    "Greater Phoenix Area": "Phoenix, AZ",
    "Greater Pittsburgh Region": "Pittsburgh, PA",
    "Greater Reno Area": "Reno, NV",
    "Greater Richmond Region": "Richmond, VA",
    "Greater Sacramento": "Sacramento, CA",
    "Greater San Luis Obispo Area": "San Luis Obispo, CA",
    "Greater Savannah Area": "Savannah, GA",
    "Greater Scranton Area": "Scranton, PA",
    "Greater Seattle Area": "Seattle, WA",
    "Greater Sioux Falls Area": "Sioux Falls, SD",
    "Greater St. Louis": "St. Louis, MO",
    "Greater Syracuse-Auburn Area": "Syracuse, NY",
    "Greater Tampa Bay Area": "Tampa, FL",
    "Greater Tucson Area": "Tucson, AZ",
    "Greater Wilmington Area": "Wilmington, DE",
    "Green Bay, Wisconsin Metropolitan Area": "Green Bay, WI",
    "Greensboro--Winston-Salem--High Point Area": "Greensboro, NC",
    "Greenville-Spartanburg-Anderson, South Carolina Area": "Greenville, SC",
    
    # H
    "Hampton Roads, Virginia Metropolitan Area": "Norfolk, VA",
    "Hilton Head Island, South Carolina Area": "Hilton Head Island, SC",
    "Honolulu Metropolitan Area": "Honolulu, HI",
    
    # J
    "Johnson City-Kingsport-Bristol Area": "Johnson City, TN",
    
    # K
    "Kansas City Metropolitan Area": "Kansas City, MO",
    "Knoxville Metropolitan Area": "Knoxville, TN",
    
    # L
    "La Crosse-Onalaska Area": "La Crosse, WI",
    "Lafayette, Indiana Metropolitan Area": "Lafayette, IN",
    "Lafayette, Louisiana Metropolitan Area": "Lafayette, LA",
    "Las Vegas Metropolitan Area": "Las Vegas, NV",
    "Lawton Area": "Lawton, OK",
    "Lincoln, Nebraska Metropolitan Area": "Lincoln, NE",
    "Little Rock Metropolitan Area": "Little Rock, AR",
    "Los Angeles Metropolitan Area": "Los Angeles, CA",
    "Louisville Metropolitan Area": "Louisville, KY",
    "Lubbock-Levelland Area": "Lubbock, TX",
    
    # M
    "Maui": "Lahaina, HI",
    "Memphis Metropolitan Area": "Memphis, TN",
    "Metro Jacksonville": "Jacksonville, FL",
    "Metropolitan Fresno": "Fresno, CA",
    "Miami-Fort Lauderdale Area": "Miami, FL",
    "Mobile Metropolitan Area": "Mobile, AL",
    "Modesto-Merced Area": "Modesto, CA",
    
    # N
    "Nashville Metropolitan Area": "Nashville, TN",
    "New Bern-Morehead City Area": "New Bern, NC",
    "New York City Metropolitan Area": "New York, NY",
    
    # O
    "Oklahoma City Metropolitan Area": "Oklahoma City, OK",
    "Omaha Metropolitan Area": "Omaha, NE",
    
    # P
    "Pensacola Metropolitan Area": "Pensacola, FL",
    "Peoria Metropolitan Area": "Peoria, IL",
    "Portland, Maine Metropolitan Area": "Portland, ME",
    "Portland, Oregon Metropolitan Area": "Portland, OR",
    "Pueblo-Cañon City Area": "Pueblo, CO",
    
    # R
    "Raleigh-Durham-Chapel Hill Area": "Raleigh, NC",
    "Rochester, New York Metropolitan Area": "Rochester, NY",
    "Rocky Mount-Wilson Area": "Rocky Mount, NC",
    
    # S
    "Salt Lake City Metropolitan Area": "Salt Lake City, UT",
    "San Antonio, Texas Metropolitan Area": "San Antonio, TX",
    "San Diego Metropolitan Area": "San Diego, CA",
    "San Francisco Bay Area": "San Francisco, CA",
    "South Bend-Mishawaka Region": "South Bend, IN",
    "Springfield, Illinois Metropolitan Area": "Springfield, IL",
    "Springfield, Massachusetts Metropolitan Area": "Springfield, MA",
    
    # T
    "Tallahassee Metropolitan Area": "Tallahassee, FL",
    "Toledo, Ohio Metropolitan Area": "Toledo, OH",
    "Topeka Metropolitan Area": "Topeka, KS",
    "Tulsa Metropolitan Area": "Tulsa, OK",
    
    # U
    "Utica-Rome Area": "Utica, NY",
    
    # W
    "Washington DC-Baltimore Area": "Washington, DC",
    "Waterloo-Cedar Falls Area": "Waterloo, IA",
    "Wichita, Kansas Metropolitan Area": "Wichita, KS",
    "Walla Walla Area" : "Walla Walla, WA",
    
    # Y
    "Youngstown-Warren area": "Youngstown, OH"
}
    
    def standardize_location(location):
        if pd.isna(location):
            return location
            
        location = str(location).strip()
        
        # First check if the location is in our metro area mapping
        if location in metro_to_city_state:
            return metro_to_city_state[location]
        
        # If location is just "United States", keep it as is
        if location == "United States":
            return location
            
        # Remove ", United States" if it exists with other information
        if ", United States" in location:
            location = location.replace(", United States", "")
        
        # Remove " County" or " county"
        location = re.sub(r' [Cc]ounty', '', location)
        
        # Check if it's a state name by itself
        if location in state_to_abbrev:
            return state_to_abbrev[location]
        
        # Check for city, state name patterns
        for state_name, abbrev in state_to_abbrev.items():
            if f", {state_name}" in location:
                # Replace state name with abbreviation
                return location.replace(f", {state_name}", f", {abbrev}")
        
        # Return the original location if no transformations apply
        return location
    
    # Apply standardization directly to the location column
    df['location'] = df['location'].apply(standardize_location)
    
    return df

postings_with_salary = standardize_locations(postings_with_salary)

In [23]:
import re
non_standard_locations = []
for loc in postings_with_salary['location'].dropna().unique():
   if not re.search(r', [A-Z]{2}$', str(loc)):
       non_standard_locations.append(loc)

# Sort and print them
non_standard_locations.sort()
print(len(non_standard_locations))

print(f"Locations without standard City, State format ({len(non_standard_locations)} found):")
for loc in non_standard_locations:
   print(f"- {loc}")

50
Locations without standard City, State format (50 found):
- AK
- AL
- AR
- AZ
- CA
- CO
- CT
- DC
- DE
- FL
- GA
- HI
- IA
- ID
- IL
- IN
- KS
- KY
- LA
- MA
- MD
- ME
- MI
- MN
- MO
- MS
- MT
- NC
- ND
- NE
- NH
- NJ
- NM
- NV
- NY
- OH
- OK
- OR
- PA
- RI
- SC
- SD
- TN
- TX
- UT
- United States
- VA
- WA
- WI
- WV


In [24]:
import pandas as pd

# Assuming you've already loaded these:
# postings = pd.read_csv('raw_linkedin_data/postings.csv')
# jobs_industries = pd.read_csv('raw_linkedin_data/jobs/job_industries.csv')
# job_skills = pd.read_csv('raw_linkedin_data/jobs/job_skills.csv')

# Convert job_id to string for consistent joining
postings_with_salary['job_id'] = postings_with_salary['job_id'].astype(str)
jobs_industries['job_id'] = jobs_industries['job_id'].astype(str)
job_skills['job_id'] = job_skills['job_id'].astype(str)

# Group industries by job_id and create a comma-separated list
jobs_industries_grouped = jobs_industries.groupby('job_id')['industry_id'].apply(
    lambda x: ','.join(map(str, x))
).reset_index()

# Group skills by job_id and create a comma-separated list
job_skills_grouped = job_skills.groupby('job_id')['skill_abr'].apply(
    lambda x: ','.join(map(str, x))
).reset_index()

# Merge postings with industries
postings_with_salary = pd.merge(
    postings_with_salary,
    jobs_industries_grouped,
    on='job_id',
    how='left'
)

# Merge with skills
postings_with_salary = pd.merge(
    postings_with_salary,
    job_skills_grouped,
    on='job_id',
    how='left'
)

# Replace NaN values with empty strings for the new columns
postings_with_salary['industry_id'] = postings_with_salary['industry_id'].fillna('')
postings_with_salary['skill_abr'] = postings_with_salary['skill_abr'].fillna('')

# Now postings has the two new columns: industry_id and skill_abr
print(f"Added industry_id and skill_abr columns to postings")
print(f"Postings shape: {postings_with_salary.shape}")

# If you want to save the enriched data
# postings.to_csv('enriched_postings.csv', index=False)

Added industry_id and skill_abr columns to postings
Postings shape: (36073, 35)


In [25]:
postings_with_salary['job_id'] = postings_with_salary['job_id'].astype(str)
benefits['job_id'] = benefits['job_id'].astype(str)

# Create dictionaries to store the separated benefits
inferred_benefits_dict = {}
non_inferred_benefits_dict = {}

# Initialize empty lists for all job_ids
for job_id in postings_with_salary['job_id'].unique():
    inferred_benefits_dict[job_id] = []
    non_inferred_benefits_dict[job_id] = []

# Get benefits that match job_ids in postings_with_salary
job_benefits = benefits[benefits['job_id'].isin(postings_with_salary['job_id'])]

# Fill the dictionaries
for _, row in job_benefits.iterrows():
    job_id = row['job_id']
    benefit_type = row['type']
    inferred = row['inferred']
    
    if inferred == 1:
        inferred_benefits_dict[job_id].append(benefit_type)
    else:  # inferred == 0
        non_inferred_benefits_dict[job_id].append(benefit_type)

# Function to convert list to comma-separated string
def list_to_comma_str(lst):
    if not lst:
        return ""
    return ",".join(lst)

# Create the new columns using the dictionaries
postings_with_salary['inferred_benefits'] = postings_with_salary['job_id'].map(
    lambda x: list_to_comma_str(inferred_benefits_dict.get(x, []))
)
postings_with_salary['non_inferred_benefits'] = postings_with_salary['job_id'].map(
    lambda x: list_to_comma_str(non_inferred_benefits_dict.get(x, []))
)

print(f"Added separated benefit columns to postings_with_salary")
print(f"Final shape: {postings_with_salary.shape}")

Added separated benefit columns to postings_with_salary
Final shape: (36073, 37)


In [26]:
# Save final_company_data to a CSV file
final_company_data.to_csv('processed_linkedin_companies.csv', index=False)

# Save postings_with_salary to a CSV file
postings_with_salary.to_csv('processed_linkedin_postings_salary.csv', index=False)